<a href="https://colab.research.google.com/github/ggongch2/web-scarping/blob/main/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 15.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#최초 1회
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller


In [ ]:
https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium
https://m.blog.naver.com/sinx2233/223223626622 # selenium 응답 확인 seleniumwire
https://velog.io/@sangyeon217/selenium-webdriver-wait webdriverwait


In [3]:
from bs4 import BeautifulSoup, Tag
import requests
import re
import random
import sys
import selenium
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

In [4]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어


In [ ]:
# 랜덤문서에 방문해서 그문서에 기여한 사람 집계
# 다른 문서에 기여한 사람 집계 -> 모두 나라별
# 그래서 나라별 기여도가 비슷할수록 비슷한 문서/연관성이 높다고 볼수 있지않을까?

# 외부링크들을 모아서 Set에 넣고
#위키 재밌는 문서 찾기
# 랜덤문서 찾고 연관 문서가 다양한 것 ? 많은것?

In [5]:
def get_href_name(url) :
  if url is not None :
    if url.startswith("https://en.wikipedia.org/wiki/") :
      parts = url.split("/")
      wiki_index = parts.index("wiki")
      if "Wikipedia" not in parts[wiki_index+1] :
        return parts[wiki_index+1]
  else :
    return None
#cite도 없어야함, category 이것도 없어야함
#나머지 중복링크는 직접 수작업으로 처리
"""
# 아래는 예시
urls = [
    "https://en.wikipedia.org/wiki/Geographic_coordinate_system",
    "https://en.wikipedia.org/wiki/New_South_Wales",
    "https://en.wikipedia.org/wiki/Main_Page",  # 정상적인 URL
    "https://en.wikipedia.org/",  # wiki가 없는 경우
    "https://www.example.com/page1", # 다른 URL
    "https://en.wikipedia.org/wiki/",
    "https://en.wikipedia.org/wiki/Wollemi_National_Park",
    "https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_4.0_International_License",
    "https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Universal_Code_of_Conduct"

]


for url in urls :
  print(get_href_name(url))
"""

'\n# 아래는 예시\nurls = [\n    "https://en.wikipedia.org/wiki/Geographic_coordinate_system",\n    "https://en.wikipedia.org/wiki/New_South_Wales",\n    "https://en.wikipedia.org/wiki/Main_Page",  # 정상적인 URL\n    "https://en.wikipedia.org/",  # wiki가 없는 경우\n    "https://www.example.com/page1", # 다른 URL\n    "https://en.wikipedia.org/wiki/",\n    "https://en.wikipedia.org/wiki/Wollemi_National_Park",\n    "https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_4.0_International_License",\n    "https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Universal_Code_of_Conduct"\n\n]\n\n\nfor url in urls :\n  print(get_href_name(url))\n'

In [28]:
#wiki 에서 #cite_note와 Category: 는 제외

not_include = ['Category:', '#cite_note', '#cite_ref', 'File:', '#bodyContent']

In [13]:
basic_title = list({'Special:RecentChangesLinked', 'Special:MyContributions', 'Main_Page', 'Special:RecentChanges', 'Special:Search', 'Special:Random', 'Special:SpecialPages', 'Help:Category', 'Help:Introduction', 'Help:Contents', 'Special:WhatLinksHere', 'Special:MyTalk', 'Portal:Current_events'})
basic_urls = ['https://en.wikipedia.org/wiki/Special:RecentChangesLinked', 'https://en.wikipedia.org/wiki/Special:MyContributions', 'https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Special:RecentChanges', 'https://en.wikipedia.org/wiki/Special:Search', 'https://en.wikipedia.org/wiki/Special:Random', 'https://en.wikipedia.org/wiki/Special:SpecialPages', 'https://en.wikipedia.org/wiki/Help:Category', 'https://en.wikipedia.org/wiki/Help:Introduction', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Special:WhatLinksHere', 'https://en.wikipedia.org/wiki/Special:MyTalk', 'https://en.wikipedia.org/wiki/Portal:Current_events']

In [30]:
def get_href_set(driver) :
  href_set = set()
  #a_tag = driver.find_elements(By.TAG_NAME, 'a')
  a_tag = driver.find_elements(By.XPATH, "//a[not(@class='vector-toc-link' or @class='mw-file-description')]")
  title = get_title(driver)
  for each in a_tag :
    href_url = each.get_attribute('href')
    href_value = get_href_name(href_url)
    if href_value is not None and href_value not in basic_title and href_value is not title :
        if not any(word in href_value for word in not_include) :
          href_set.add(href_value)
  return href_set

# href 링크 모으기. class=new인건 수집x. /wiki/... 형식이여야함



In [8]:
def random_wiki_page() :
  url = "https://en.wikipedia.org/wiki/Special:Random"
  driver = webdriver.Chrome(options = chrome_options)
  driver.get(url)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  return driver

In [9]:
def get_title(driver) :
  return driver.find_element(By.ID, 'firstHeading').text


In [26]:
def how_many_href(driver) :
  a_tag_set = get_href_set(driver)
  return len(a_tag_set)
  # a_tag 수를 알려면 h_set을 알아야하고, get_href_set을 여러번 돌려야한다는 것을 안다면, 매우 비효율적인 함수
  # 자손을 알때는 유용한데 이미 주어진경우는 h_set을 저장해두는 것이 효율적일수도
  # 전체 h_ref_list에 dict 원소로, key:문서제목, value:href_set으로 저장
  # 아예 class로 문서, href_set을 저장할수있지 않을까?, 메서드는 한문서에서 다른문서로 넘어가는거로

In [16]:
def make_href_url(href_value) :
  return "https://en.wikipedia.org/wiki/"+href_value

In [17]:
# key로 href_set에서 key:링크 이름, value : how_many_href(url) 로 해보기
#   get_title로 아예 key 이름을 해버리기?

In [31]:
driver = random_wiki_page()

start_href = get_title(driver)

h_set = get_href_set(driver)
h_dict = dict()

h_dict[start_href] = how_many_href(driver)

for each in h_set :
  driver.get(make_href_url(each))
  h_dict[each] = how_many_href(driver)

print(h_dict)

{'Karimabad, Andika': 337, 'Sefiddara': 334, 'Sar_Bazar': 335, 'Chegarman': 335, 'Sar_Murd-e_Isi': 332, 'Talkhab-e_Khun_Ali': 332, 'Tina,_Iran': 334, 'Chal-e_Balutak': 333, 'Bahramabad,_Abezhdan': 334, 'Qaleh_Lut': 335, 'Hasan_Ali_Darreh_Forkh': 332, 'Alma-ye_Sofla': 334, 'Pa_Ab-e_Shelal': 426, 'Chuzu': 334, 'UTC%2B3:30': 95, 'Kamarsheh_Seyyedi_Dallah': 333, 'Gerdab,_Khuzestan': 337, 'Tachel_Ali_Shir': 332, 'Rural_Districts_of_Iran': 302, 'Gurab,_Andika': 335, 'Andikayi': 333, 'Murgeh': 333, 'Sarchenar,_Khuzestan': 334, 'Rafiabad,_Khuzestan': 336, 'Template:Andika_County': 317, 'Gholam_Hoseyn-e_Hafezi': 334, 'Sar_Gach,_Andika': 335, 'Boneh-ye_Musa': 332, 'Ab_Chendar,_Abezhdan': 334, 'Alidad,_Khuzestan': 332, 'Hamisheh_Behar': 332, 'Ab_Kabud-e_Sofla': 333, 'Adiv_Chaliv': 333, 'Muri,_Khuzestan': 334, 'Seyyed_Rajab': 333, 'Taherabad,_Andika': 425, 'Ahmadabad-e_Dinarak': 336, 'Sukhteh': 334, 'Harbazchellow': 334, 'Chezi-ye_Olya': 334, 'Emamzadeh_Shahzadeh_Abdollah': 338, 'Template_talk:And

In [ ]:
def get_rNode_info :
  driver = random_wiki_page()

  start_href = get_title(driver)


  h_set = get_href_set(driver)
  h_dict = dict()

  h_dict[start_href] = how_many_href(driver)

  for each in h_set :
    driver.get(make_href_url(each))
    h_dict[each] = how_many_href(driver)

  print(h_dict)

In [39]:
  # 아예 class로 그래프 형태로 저장하여 문서, href_set을 저장할수있지 않을까?, 메서드는 한문서에서 다른문서로 넘어가는거로 <- 너무 범위가 커지는듯
  #차라리 문서 하나를 객체로 해서 href_set과 title을 관리?
  # get_url을 할때마다 class에 접근해서 해당 객체가 존재하는 경우 그대로 가져다씀
  # 위키 추천시스템? <- 최종계획 ( 문서의 양, 링크의수, 연결된 링크들의 링크 수)

class Node :
    def __init__(self, title, href_set) :
      self.title = title
      self.href_set = href_set
  #만약 해당하는 이름의 title을 가진 객체가 있을경우 href_set을 return
    def get_href_set(self) :
      return self.href_set
    def get_title(self) :
      return self.title
